# Controlador de vuelo para vehículos aéreos no tripulados multi-rotor basado en técnicas de aprendizaje profundo

## Entrenamiento Red LSTM

### Javier Cárdenas - Uriel Carrero

## 1. Descripción del Dataset

#### Importar Librerías

In [1]:
import os
import sys
import random

import pandas as pd
pd.set_option('display.max_columns', None) #Para mostrar todas las columnas
import matplotlib.pyplot as plt

import numpy as np                  # Cómputo Numérico
print(np.__version__)
#assert (np.__version__=='1.19.5'), 'Versión incorrecta de numpy, por favor instale 1.19.5'
seed = 5
np.random.seed(seed)
np.seterr(divide = 'ignore')

1.22.2


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
os.environ['TF_KERAS'] = '1'
import keras as kr
import keras_tuner as kt

import tensorflow as tf
from tensorflow.keras import models, layers
print(tf.__version__)
#assert (tf.__version__=='2.5.0'), 'Versión incorrecta de Tensorflow, por favor instale 2.5.0'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from IPython.display import clear_output

2.8.0
Num GPUs Available:  1


2022-02-08 19:59:19.459667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 19:59:19.495880: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 19:59:19.496089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
print(kr.__version__)

2.8.0


In [4]:
gpus = tf.config.list_physical_devices('GPU')
config = ConfigProto()
if gpus:
    try:
        config.gpu_options.allow_growth = True
        tf.compat.v1.enable_eager_execution()

        os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
session = InteractiveSession(config=config)

1 Physical GPUs, 1 Logical GPUs


2022-02-08 19:59:19.514827: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-08 19:59:19.516803: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 19:59:19.517126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-08 19:59:19.517366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [5]:
import gc #garbage collector
import gc; gc.enable()

## Cargar Datos

### Leemos el Dataset

In [6]:
root = '../logs/Datasets/'
dataset_name = 'DatasetXYZYaw_2022V3'
rootdir = root+dataset_name
if not os.path.exists(rootdir):
    print(f"{rootdir} not exist")

In [7]:
df = pd.read_csv(os.path.join(rootdir, random.choice(os.listdir(rootdir))))
delete_list =['Time', 'Bat_V','Bat_Percentage','P_r', 'Q_r', 'R_r', 'Flag_Pitch_Roll', 'Gyro P', 'Gyro Q', 'Gyro R','Ac_Dx', 'Ac_Dy', 'Ac_Dz','Sonar Altitud','Pressure Altitud', 'Acceleracion P', 'Acceleracion Q', 'Acceleracion R', 'Pitch_r', 'Roll_r']
df_list = df.columns.to_list()
rpm_list = [i for i in df_list if ("Motor" in i)]
states_list = [i for i in df_list if not ((i in delete_list) or (i in rpm_list))]
print(states_list)

['X', 'Y', 'Z', 'Yaw', 'Roll', 'Pitch', 'Dx', 'Dy', 'Dz', 'P', 'Q', 'R', 'X_r', 'Y_r', 'Z_r', 'Yaw_r', 'Dx_r', 'Dy_r', 'Dz_r', 'Acceleracion X', 'Acceleracion Y', 'Acceleracion Z']


In [8]:
dataset = []
for filename in os.listdir(rootdir):
    if not filename.endswith(".csv"):
        continue
        
    df = pd.read_csv(os.path.join(rootdir, filename))
    df =  df.drop(delete_list, axis=1)
    x = df.drop(rpm_list, axis=1)
    y = df.drop(states_list, axis=1)
    dataset.append([x, y])

df = None
x = None
y= None

#### Normalización de Estados (Entradas) y Acciones (Salidas)

In [9]:
def Norm(df, df_desc):
    for prop in list(df.columns):
        try:
            # 1 ~ Mean  7 ~ Max  3 ~ Min
            df[prop] = (df[prop]-df_desc[prop]['mean'])/(df_desc[prop]['max']-df_desc[prop]['min'])
        except:
            pass
    return df

In [10]:
norm_data_path = f"{root}/data_description_{dataset_name}.csv"
df_desc = pd.read_csv(norm_data_path, index_col=0)  
df_desc

,X,Y,Z,Yaw,Roll,Pitch,Dx,Dy,Dz,P,Q,R,Motor1,Motor2,Motor3,Motor4,X_r,Y_r,Z_r,Yaw_r,Dx_r,Dy_r,Dz_r,Acceleracion X,Acceleracion Y,Acceleracion Z
count,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06,3.643359e+06
mean,-2.784352e-01,2.571791e-02,-1.397319e+00,6.962940e-02,-2.971250e-04,2.132265e-03,-2.570198e-03,-2.440764e-03,-7.117861e-03,6.390789e-05,3.007596e-05,1.656033e-03,2.364390e+02,-2.361374e+02,2.363083e+02,-2.357950e+02,-2.944603e-01,3.445105e-02,-1.373427e+00,6.981635e-02,-3.537395e-03,-6.828863e-04,-1.179497e-02,1.581646e-05,3.663370e-05,-1.120612e-04
std,1.240217e+00,1.173091e+00,3.830872e-01,9.966686e-01,6.502062e-02,7.080256e-02,2.966397e-01,2.622465e-01,4.809884e-01,2.741369e-01,1.661207e-01,2.591436e-01,5.096122e+01,5.077905e+01,5.100095e+01,5.085239e+01,1.201457e+00,1.231739e+00,3.681867e-01,1.001667e+00,1.611526e+00,1.648572e+00,2.889408e+00,1.817114e+00,1.693403e+00,1.785579e+00
min,-1.231915e+01,-7.090483e+00,-3.332850e+00,-4.438538e+00,-1.324996e+00,-1.137046e+00,-9.357576e+00,-9.413551e+00,-4.525061e+00,-5.813348e+00,-5.234743e+00,-6.024277e+00,1.000000e+01,-5.000000e+02,1.000000e+01,-5.000000e+02,-5.000000e+00,-3.900000e+00,-2.700000e+00,-4.438500e+00,-1.000000e+03,-7.800000e+02,-4.800000e+02,-1.264318e+02,-1.220812e+02,-1.185523e+02
25%,-6.970176e-01,-4.948882e-01,-1.604458e+00,-2.847272e-05,-5.433369e-04,-7.611169e-04,-2.167348e-03,-2.540649e-03,-9.465619e-03,-3.752266e-03,-2.123620e-03,-1.113754e-03,2.352453e+02,-2.380045e+02,2.352170e+02,-2.379442e+02,-7.200000e-01,-5.000000e-01,-1.600000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.449409e-02,-7.465180e-02,-1.378055e-01
50%,-1.530501e-02,-2.376076e-03,-1.305322e+00,1.971090e-06,8.764859e-05,3.983907e-05,4.402902e-05,-8.569541e-05,5.088306e-04,1.883331e-05,-5.639626e-06,-1.172069e-05,2.365541e+02,-2.365703e+02,2.365514e+02,-2.365388e+02,0.000000e+00,0.000000e+00,-1.300000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.330169e-03
75%,1.223126e-02,4.957303e-02,-1.205790e+00,4.103759e-05,6.933445e-04,8.392792e-04,2.196593e-03,2.449358e-03,1.023162e-02,3.842387e-03,2.106913e-03,1.099744e-03,2.378871e+02,-2.351854e+02,2.379059e+02,-2.351264e+02,0.000000e+00,2.000000e-02,-1.202435e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.436232e-02,7.632254e-02,1.315630e-01
max,7.269514e+00,7.684191e+00,-3.355866e-02,5.520342e+00,1.097143e+00,1.330709e+00,8.122461e+00,7.997578e+00,5.117261e+00,6.528901e+00,6.616588e+00,6.042552e+00,5.000000e+02,-1.000000e+01,5.000000e+02,-1.000000e+01,4.100000e+00,4.400000e+00,0.000000e+00,5.519600e+00,8.200000e+02,8.800000e+02,1.999644e+02,1.265200e+02,1.247206e+02,1.427007e+02


In [11]:
for i, data in enumerate(dataset):
    x, y = data
    Norm(x, df_desc)
    Norm(y, df_desc)
    dataset[i]=[x,y]

In [12]:
data[0].describe()

,X,Y,Z,Yaw,Roll,Pitch,Dx,Dy,Dz,P,Q,R,X_r,Y_r,Z_r,Yaw_r,Dx_r,Dy_r,Dz_r,Acceleracion X,Acceleracion Y,Acceleracion Z
count,24001.000000,24001.000000,24001.000000,24001.000000,24001.000000,24001.000000,24001.000000,24001.000000,24001.000000,24001.000000,2.400100e+04,24001.000000,24001.000000,24001.000000,24001.000000,2.400100e+04,24001.000000,2.400100e+04,24001.000000,2.400100e+04,2.400100e+04,2.400100e+04
mean,-0.042378,-0.066094,-0.081814,-0.006989,-0.000142,-0.000841,-0.000536,-0.000585,-0.000337,-0.000003,-2.951640e-06,-0.000120,-0.090035,-0.119666,-0.096883,-7.011011e-03,-0.000007,-7.469821e-06,-0.000006,-1.573812e-08,-2.931281e-07,2.872697e-08
std,0.027813,0.035779,0.098353,0.000125,0.007912,0.016093,0.024912,0.008713,0.042029,0.007587,1.976457e-02,0.000431,0.058884,0.063297,0.115714,1.734760e-18,0.000582,6.093959e-04,0.002963,1.176002e-02,4.342281e-03,5.641351e-03
min,-0.087236,-0.125356,-0.582363,-0.009163,-0.054021,-0.162878,-0.238682,-0.072742,-0.468485,-0.172445,-1.829388e-01,-0.003312,-0.173136,-0.193307,-0.257990,-7.011011e-03,-0.054943,-5.783091e-02,-0.382356,-2.972453e-01,-9.008613e-02,-4.145890e-02
25%,-0.069954,-0.105570,-0.139723,-0.006993,-0.000400,-0.001022,0.000034,-0.000026,-0.000051,-0.000286,-1.734974e-04,-0.000220,-0.148960,-0.189693,-0.176509,-7.011011e-03,0.000002,4.113773e-07,0.000017,-5.996631e-05,-3.958466e-04,-4.880695e-04
50%,-0.038492,-0.079476,-0.112906,-0.006991,-0.000043,-0.000822,0.000147,0.000131,0.000800,-0.000004,-3.770483e-09,-0.000140,-0.080829,-0.143910,-0.146879,-7.011011e-03,0.000002,4.113773e-07,0.000017,-6.707869e-07,2.562496e-06,-6.364915e-06
75%,-0.019401,-0.031909,-0.015868,-0.006989,0.000226,-0.000567,0.000265,0.000301,0.001665,0.000286,1.734929e-04,-0.000060,-0.040169,-0.086078,-0.028361,-7.011011e-03,0.000002,4.113773e-07,0.000017,5.829777e-05,4.025907e-04,4.723632e-04
max,0.023469,0.003642,0.410454,-0.005749,0.040394,0.099488,0.230330,0.039661,0.344611,0.086610,1.934703e-01,0.008400,0.032358,-0.004151,0.508677,-7.011011e-03,0.000002,4.113773e-07,0.100023,1.209756e-01,9.736804e-02,5.645216e-02


In [13]:
data[1].describe()

,Motor1,Motor2,Motor3,Motor4
count,24001.000000,24001.000000,24001.000000,24001.000000
mean,-0.007031,0.006472,-0.006740,0.005988
std,0.074874,0.075434,0.074752,0.074797
min,-0.462120,-0.406565,-0.461854,-0.313352
25%,-0.002066,-0.002872,-0.001841,-0.003564
50%,0.000094,-0.000707,0.000352,-0.001378
75%,0.002146,0.001555,0.002444,0.000881
max,0.311689,0.461505,0.351106,0.460806


#### División del dataset para entrenamiento, validación, prueba

In [14]:
X = []
Y = []

for sample in dataset:
    x, y = sample
    X.append(x)
    Y.append(y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.05, random_state=42)
X_train,  X_val, Y_train, Y_val  = train_test_split(X_train, Y_train, test_size=0.1)

del X
del Y

In [15]:
print(f'Total={len(dataset)}, Entrenamiento={len(X_train)} ({round(100*len(X_train)/len(dataset))}%),'\
      f'Validación={len(X_val)} ({round(100*len(X_val)/len(dataset))}%),'\
      f'Prueba={len(X_test)} ({round(100*len(X_test)/len(dataset))}%)')

Total=159, Entrenamiento=135 (85%),Validación=16 (10%),Prueba=8 (5%)


#### Generador de Ejemplos de entrenamiento

Entrenar un modelo con una señal de 50000 datos en cada iteración sería una tarea que tomaría demasiado tiempo, así mismo, cuando se necesite realizar la inferencia del modelo, se necesitaría esa misma cantidad de datos, por lo que no se utiliza toda la señal de entrenamiento, sino pequeños segmentos de tamaño N, por lo que se generarán M-N (longitud de toda la señal, 50000) señales de longitud N para el entrenamiento, lo que aumentaría el consumo de memoria. Por tal motivo se define un generador.

In [16]:
class DataGenerator:
    def __init__(self, X=[], Y=[], dataset = None, batch_size=512, window=512, sequence_out=False, variable_window=False, delta_window=1, feedback=False, window_feedback=1):
        if dataset:
            for data in dataset:
                X.append(data[0])
                Y.append(data[1])
            self.X = X
            self.Y = Y
        elif X and Y:
            if len(X)!=len(Y):
                raise Exception("La longitud de datos de X e Y deben ser iguales")
            self.X = X
            self.Y = Y
        else:
            raise Exception("Debe especificar dataset o X, Y")
            
        self.n  = len(X)                 ### Número de ejemplos de entrenamiento
        x_shape = X[0].shape
        y_shape = Y[0].shape
        self.batch_size = batch_size
        self.window = window
        self.variable_window = variable_window
        self.delta_window = delta_window
        self.feedback = feedback
        self.i  = x_shape[1] if not self.feedback else x_shape[1]+y_shape[1]   ### Número de características
        self.j  = y_shape[1]                                                   ### Número de salidas
        #self.window_feedback = window_feedback
        if self.variable_window:
            self.window_max = self.window+self.delta_window
            self.window_min = self.window-self.delta_window
            if self.window_min<1:
                raise IndexError(f'delta_window no puede ser igual o mayor a la ventana')
        self.sequence_out = sequence_out
        self.set_shapes()
        
    def set_shapes(self):
        if self.sequence_out:
            self.shapes = ((self.batch_size, self.window, self.i),
                           (self.batch_size, self.window, self.j))
        else:
            self.shapes = ((self.batch_size, self.window, self.i),
                           (self.batch_size, self.j))

    def buid_init(self):
        if self.variable_window:
            self.window = np.random.randint(self.window_min, self.window_max)
            self.set_shapes()
        self.samples = np.empty(shape= self.shapes[0], dtype='float32') 
        self.labels  = np.empty(shape= self.shapes[1], dtype='float32') 
        self.batchcount = 0
        
    def build_data(self):
        self.buid_init()
        if self.feedback:
            i_0 = 1
        else:
            i_0 = 0
        while True:
            try:
                index = np.random.randint(0, self.n-1)            ### Trayectoria a seleccionar
                m = len(self.X[index])          ### Número de steps por ejemplo
                if m-self.window-1<=0:
                    raise IndexError(f'El tamaño de la ventana es mayor a la trayectoria')
                else:
                    start_index = np.random.randint(i_0, int(m-self.window-1))
                    final_index = start_index+self.window
                    x = self.X[index][start_index:final_index].to_numpy()
                    if self.feedback:
                        y = self.Y[index][start_index-1:final_index-1].to_numpy()
                        self.samples[self.batchcount] = np.concatenate((x,y), axis=1)
                    else:
                        self.samples[self.batchcount] = x
                        
                    if self.sequence_out:
                        self.labels[self.batchcount] = self.Y[index][start_index:final_index].to_numpy()
                    else:
                        self.labels[self.batchcount] = self.Y[index].loc[final_index]
            except IndexError as e:
                print(f'ERROR: Ejemplo {self.batchcount}: {e}')
                raise e
                
            self.batchcount += 1
            if self.batchcount >= self.batch_size:
                yield self.samples.astype(np.float32), self.labels.astype(np.float32)
                self.buid_init()

In [17]:
window = 64                   ### Número de steps por ejemplo
batch_size = 512              ### Número de ejemplos por batch
sequence_out = False
variable_window=False
feedback = False
delta_window=window/3

In [18]:
train_generator = DataGenerator(X=X_train, Y=Y_train, batch_size=batch_size, window=window, sequence_out=sequence_out, variable_window=variable_window, delta_window=delta_window, feedback=feedback)
val_generator   = DataGenerator(X=X_val,   Y=Y_val,   batch_size=batch_size, window=window, sequence_out=sequence_out, variable_window=variable_window, delta_window=delta_window, feedback=feedback)
test_generator  = DataGenerator(X=X_test,  Y=Y_test,  batch_size=batch_size, window=window, sequence_out=sequence_out, variable_window=variable_window, delta_window=delta_window, feedback=feedback)

In [19]:
input_dim = len(states_list) #if not feedback else len(states_list)+len(rpm_list)
output_dim = len(rpm_list)
print(f'input_dim: {input_dim}, output_dim: {output_dim}')

input_dim: 22, output_dim: 4


In [20]:
dataset_train = tf.data.Dataset.from_generator(train_generator.build_data,
                                        output_types = (tf.float32, tf.float32), 
                                        output_shapes=(tf.TensorShape((None, window, input_dim)), tf.TensorShape((None, output_dim))))
dataset_val = tf.data.Dataset.from_generator(val_generator.build_data,
                                        output_types = (tf.float32, tf.float32),
                                        output_shapes=(tf.TensorShape((None, window, input_dim)), tf.TensorShape((None, output_dim))))
dataset_test = tf.data.Dataset.from_generator(test_generator.build_data,
                                        output_types = (tf.float32, tf.float32),
                                        output_shapes=(tf.TensorShape((None, window, input_dim)), tf.TensorShape((None, output_dim))))

In [21]:
for _ in range(5):
    x, y = next(train_generator.build_data())
    print(f'x.shape={x.shape}, y.shape={y.shape}')

x.shape=(512, 64, 22), y.shape=(512, 4)
x.shape=(512, 64, 22), y.shape=(512, 4)
x.shape=(512, 64, 22), y.shape=(512, 4)
x.shape=(512, 64, 22), y.shape=(512, 4)
x.shape=(512, 64, 22), y.shape=(512, 4)


## Keras Model

## Callbacks

In [22]:
main_metric = 'mean_squared_error'
#metrics = [main_metric, 'cosine_similarity', 'logcosh']
metrics = main_metric

#### Definición del Modelo

In [23]:
input_dim = len(states_list) if not feedback else len(states_list)+len(rpm_list)
output_dim = len(rpm_list)
print(f'input_dim: {input_dim}, output_dim: {output_dim}')

input_dim: 22, output_dim: 4


In [24]:
class LSTMHyperModel(kt.HyperModel):
    def __init__(self, input_dim, output_dim, metrics='mean_squared_error', loss='mean_squared_error'):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.metrics = metrics
        self.loss = loss

    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(layers.LSTM(units=hp.Int('LSTM_units',min_value=8,max_value=512,step=16,default=256),
                              input_shape=(None, self.input_dim), return_sequences=True))
        
        hidden_LSTMlayers = hp.Int('Hidden_LSTMlayers',min_value=1,max_value=5,step=1,default=5)
        for i in range(hidden_LSTMlayers):
            if i==hidden_LSTMlayers-1:
                return_sequences = False
            else:
                return_sequences = True
                
            model.add(layers.LSTM(units=hp.Int(f'LSTM_Hidden_units_{i}',min_value=8,max_value=256,step=16,default=128),
                                  return_sequences=return_sequences))
        
        hidden_layers = hp.Int('Hidden_FClayers',min_value=1,max_value=10,step=1,default=5)
        for i in range(hidden_layers):
            model.add(layers.Dense(units=hp.Int(f'Hidden_units_{i}',min_value=8,max_value=512,step=16,default=128), 
                                   activation=hp.Choice(
                                        'dense_activation',
                                        values=['relu', 'tanh', 'sigmoid'],
                                        default='relu')))
        
        model.add(layers.Dense(self.output_dim))

        model.compile(
            optimizer=tf.keras.optimizers.Adam(
                learning_rate=hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
                )
            ),
            loss=self.loss,
            metrics=self.metrics
        )
        return model

In [25]:
hypermodel = LSTMHyperModel(input_dim, output_dim)

#### Tuner

In [26]:
SEED = 1 
MAX_TRIALS = 50
EXECUTION_PER_TRIAL = 2
HYPERBAND_MAX_EPOCHS = 150

tuner = kt.tuners.Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective=F'val_{main_metric}',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='./Models/hyperband/',
    project_name=f'{dataset_name}'
)

#### Compilado el Modelo

In [27]:
tuner.search_space_summary()

Search space summary
Default search space size: 15
LSTM_units (Int)
{'default': 256, 'conditions': [], 'min_value': 8, 'max_value': 512, 'step': 16, 'sampling': None}
Hidden_LSTMlayers (Int)
{'default': 5, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
LSTM_Hidden_units_0 (Int)
{'default': 128, 'conditions': [], 'min_value': 8, 'max_value': 256, 'step': 16, 'sampling': None}
LSTM_Hidden_units_1 (Int)
{'default': 128, 'conditions': [], 'min_value': 8, 'max_value': 256, 'step': 16, 'sampling': None}
LSTM_Hidden_units_2 (Int)
{'default': 128, 'conditions': [], 'min_value': 8, 'max_value': 256, 'step': 16, 'sampling': None}
LSTM_Hidden_units_3 (Int)
{'default': 128, 'conditions': [], 'min_value': 8, 'max_value': 256, 'step': 16, 'sampling': None}
LSTM_Hidden_units_4 (Int)
{'default': 128, 'conditions': [], 'min_value': 8, 'max_value': 256, 'step': 16, 'sampling': None}
Hidden_FClayers (Int)
{'default': 5, 'conditions': [], 'min_value': 1, 'max_value': 5, 'st

#### Entrenamiento del Modelo

In [32]:
%%time
N_EPOCH_SEARCH = 150

#max_queue_size = maximum size of the internal training queue which is used to "precache" samples from the generator
#workers = number of threads generating batches in parallel. Batches are computed in parallel on the CPU and passed on the fly onto the GPU for neural network computations
tuner.search(dataset_train.repeat(),
            steps_per_epoch=len(X_train), # steps per epoch
            epochs=N_EPOCH_SEARCH,
            validation_data=dataset_val.repeat(),
            validation_steps=10,
            max_queue_size=2, ##
            workers=2,
            callbacks=[kr.callbacks.TensorBoard(f'../Models/tmp/tb_logs/{dataset_name}')]
            )

Trial 255 Complete [02h 23m 07s]
val_mean_squared_error: 0.0002698843600228429

Best val_mean_squared_error So Far: 0.00021095942793181166
Total elapsed time: 10h 22m 33s
INFO:tensorflow:Oracle triggered exit
CPU times: user 2d 8h 48min 19s, sys: 42min 16s, total: 2d 9h 30min 35s
Wall time: 2d 10h 22min


#### Se guarda el Modelo

In [33]:
# Show a summary of the search
tuner.results_summary()

Results summary
Results in ./Models/hyperband/DatasetXYZYaw_2022V3
Showing 10 best trials
Objective(name='val_mean_squared_error', direction='min')
Trial summary
Hyperparameters:
LSTM_units: 216
Hidden_LSTMlayers: 1
LSTM_Hidden_units_0: 168
LSTM_Hidden_units_1: 248
LSTM_Hidden_units_2: 216
LSTM_Hidden_units_3: 216
LSTM_Hidden_units_4: 248
Hidden_FClayers: 1
Hidden_units_0: 280
dense_activation: tanh
Hidden_units_1: 168
Hidden_units_2: 168
Hidden_units_3: 392
Hidden_units_4: 312
learning_rate: 0.00492210572872919
tuner/epochs: 150
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.00021095942793181166
Trial summary
Hyperparameters:
LSTM_units: 440
Hidden_LSTMlayers: 1
LSTM_Hidden_units_0: 184
LSTM_Hidden_units_1: 88
LSTM_Hidden_units_2: 88
LSTM_Hidden_units_3: 40
LSTM_Hidden_units_4: 8
Hidden_FClayers: 3
Hidden_units_0: 504
dense_activation: tanh
Hidden_units_1: 472
Hidden_units_2: 168
Hidden_units_3: 280
Hidden_units_4: 488
learning_rate: 0.0001617185017462098
tuner/epochs

In [34]:
EPOCHS = 150
bestHPs = tuner.get_best_hyperparameters(5)
models = []
logs = []
I = 'Tuner'
for i, bestHP in enumerate(bestHPs):
    model = tuner.hypermodel.build(bestHP)
    history = model.fit(  dataset_train.repeat(), 
                epochs=EPOCHS, 
                steps_per_epoch = len(X_train),
                verbose=1,
                validation_data = dataset_val.repeat(),
                validation_steps= len(X_val)
             )
    model.save(f'../Models/{dataset_name}_{I}_{i}.h5',  include_optimizer=False)
    print(f'../Models/{dataset_name}_{I}_{i}.h5')
    models.append(model)
    logs.append(history)

Epoch 1/150
135/135 [==============================] - 29s 189ms/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0016 - val_mean_squared_error: 0.0016
Epoch 2/150
135/135 [==============================] - 25s 182ms/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 3/150
135/135 [==============================] - 24s 179ms/step - loss: 9.6415e-04 - mean_squared_error: 9.6415e-04 - val_loss: 0.0024 - val_mean_squared_error: 0.0024
Epoch 4/150
135/135 [==============================] - 24s 179ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - val_loss: 0.0023 - val_mean_squared_error: 0.0023
Epoch 5/150
135/135 [==============================] - 25s 182ms/step - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0020 - val_mean_squared_error: 0.0020
Epoch 6/150
135/135 [==============================] - 24s 178ms/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0021 - val_mean_squared_error: 0.0

Epoch 47/150
135/135 [==============================] - 24s 180ms/step - loss: 1.6059e-04 - mean_squared_error: 1.6059e-04 - val_loss: 3.4480e-04 - val_mean_squared_error: 3.4480e-04
Epoch 48/150
135/135 [==============================] - 24s 179ms/step - loss: 1.4107e-04 - mean_squared_error: 1.4107e-04 - val_loss: 3.5796e-04 - val_mean_squared_error: 3.5796e-04
Epoch 49/150
135/135 [==============================] - 24s 180ms/step - loss: 1.7089e-04 - mean_squared_error: 1.7089e-04 - val_loss: 2.9841e-04 - val_mean_squared_error: 2.9841e-04
Epoch 50/150
135/135 [==============================] - 24s 179ms/step - loss: 1.7995e-04 - mean_squared_error: 1.7995e-04 - val_loss: 3.9355e-04 - val_mean_squared_error: 3.9355e-04
Epoch 51/150
135/135 [==============================] - 24s 179ms/step - loss: 1.8740e-04 - mean_squared_error: 1.8740e-04 - val_loss: 3.8183e-04 - val_mean_squared_error: 3.8183e-04
Epoch 52/150
135/135 [==============================] - 24s 177ms/step - loss: 1.5295

Epoch 92/150
135/135 [==============================] - 24s 177ms/step - loss: 1.4549e-04 - mean_squared_error: 1.4549e-04 - val_loss: 2.9928e-04 - val_mean_squared_error: 2.9928e-04
Epoch 93/150
135/135 [==============================] - 24s 177ms/step - loss: 1.2872e-04 - mean_squared_error: 1.2872e-04 - val_loss: 3.2191e-04 - val_mean_squared_error: 3.2191e-04
Epoch 94/150
135/135 [==============================] - 24s 178ms/step - loss: 1.2273e-04 - mean_squared_error: 1.2273e-04 - val_loss: 3.0714e-04 - val_mean_squared_error: 3.0714e-04
Epoch 95/150
135/135 [==============================] - 24s 178ms/step - loss: 1.4300e-04 - mean_squared_error: 1.4300e-04 - val_loss: 3.8188e-04 - val_mean_squared_error: 3.8188e-04
Epoch 96/150
135/135 [==============================] - 24s 177ms/step - loss: 1.6132e-04 - mean_squared_error: 1.6132e-04 - val_loss: 3.1277e-04 - val_mean_squared_error: 3.1277e-04
Epoch 97/150
135/135 [==============================] - 24s 177ms/step - loss: 1.1044

135/135 [==============================] - 24s 178ms/step - loss: 1.2848e-04 - mean_squared_error: 1.2848e-04 - val_loss: 4.0924e-04 - val_mean_squared_error: 4.0924e-04
Epoch 137/150
135/135 [==============================] - 24s 177ms/step - loss: 1.2770e-04 - mean_squared_error: 1.2770e-04 - val_loss: 3.9904e-04 - val_mean_squared_error: 3.9904e-04
Epoch 138/150
135/135 [==============================] - 24s 178ms/step - loss: 1.8756e-04 - mean_squared_error: 1.8756e-04 - val_loss: 4.7540e-04 - val_mean_squared_error: 4.7540e-04
Epoch 139/150
135/135 [==============================] - 24s 178ms/step - loss: 1.4675e-04 - mean_squared_error: 1.4675e-04 - val_loss: 3.6757e-04 - val_mean_squared_error: 3.6757e-04
Epoch 140/150
135/135 [==============================] - 24s 177ms/step - loss: 1.2953e-04 - mean_squared_error: 1.2953e-04 - val_loss: 3.2034e-04 - val_mean_squared_error: 3.2034e-04
Epoch 141/150
135/135 [==============================] - 24s 178ms/step - loss: 1.4525e-04 - m

Epoch 32/150
135/135 [==============================] - 27s 201ms/step - loss: 3.6960e-04 - mean_squared_error: 3.6960e-04 - val_loss: 5.4264e-04 - val_mean_squared_error: 5.4264e-04
Epoch 33/150
135/135 [==============================] - 27s 201ms/step - loss: 3.4223e-04 - mean_squared_error: 3.4223e-04 - val_loss: 4.9153e-04 - val_mean_squared_error: 4.9153e-04
Epoch 34/150
135/135 [==============================] - 27s 202ms/step - loss: 3.5409e-04 - mean_squared_error: 3.5409e-04 - val_loss: 4.6760e-04 - val_mean_squared_error: 4.6760e-04
Epoch 35/150
135/135 [==============================] - 27s 201ms/step - loss: 3.2379e-04 - mean_squared_error: 3.2379e-04 - val_loss: 5.3415e-04 - val_mean_squared_error: 5.3415e-04
Epoch 36/150
135/135 [==============================] - 27s 204ms/step - loss: 3.2117e-04 - mean_squared_error: 3.2117e-04 - val_loss: 3.6270e-04 - val_mean_squared_error: 3.6270e-04
Epoch 37/150
135/135 [==============================] - 27s 203ms/step - loss: 3.1732

...

Epoch 93/150
135/135 [==============================] - 35s 257ms/step - loss: 2.5919e-04 - mean_squared_error: 2.5919e-04 - val_loss: 3.0778e-04 - val_mean_squared_error: 3.0778e-04
Epoch 94/150
135/135 [==============================] - 34s 254ms/step - loss: 1.8073e-04 - mean_squared_error: 1.8073e-04 - val_loss: 2.8780e-04 - val_mean_squared_error: 2.8780e-04
Epoch 95/150
135/135 [==============================] - 34s 255ms/step - loss: 1.9445e-04 - mean_squared_error: 1.9445e-04 - val_loss: 2.7980e-04 - val_mean_squared_error: 2.7980e-04
Epoch 96/150
135/135 [==============================] - 33s 247ms/step - loss: 1.7705e-04 - mean_squared_error: 1.7705e-04 - val_loss: 3.1208e-04 - val_mean_squared_error: 3.1208e-04
Epoch 97/150
135/135 [==============================] - 34s 252ms/step - loss: 1.6804e-04 - mean_squared_error: 1.6804e-04 - val_loss: 3.4047e-04 - val_mean_squared_error: 3.4047e-04
Epoch 98/150
135/135 [==============================] - 34s 255ms/step - loss: 1.4181

135/135 [==============================] - 34s 255ms/step - loss: 1.2411e-04 - mean_squared_error: 1.2411e-04 - val_loss: 3.7368e-04 - val_mean_squared_error: 3.7368e-04
Epoch 138/150
135/135 [==============================] - 35s 256ms/step - loss: 1.2756e-04 - mean_squared_error: 1.2756e-04 - val_loss: 3.6791e-04 - val_mean_squared_error: 3.6791e-04
Epoch 139/150
135/135 [==============================] - 34s 253ms/step - loss: 1.5410e-04 - mean_squared_error: 1.5410e-04 - val_loss: 3.0723e-04 - val_mean_squared_error: 3.0723e-04
Epoch 140/150
135/135 [==============================] - 34s 254ms/step - loss: 1.7769e-04 - mean_squared_error: 1.7769e-04 - val_loss: 2.7183e-04 - val_mean_squared_error: 2.7183e-04
Epoch 141/150
135/135 [==============================] - 33s 242ms/step - loss: 1.1770e-04 - mean_squared_error: 1.1770e-04 - val_loss: 3.5729e-04 - val_mean_squared_error: 3.5729e-04
Epoch 142/150
135/135 [==============================] - 33s 246ms/step - loss: 1.3261e-04 - m

135/135 [==============================] - 29s 212ms/step - loss: 0.0092 - mean_squared_error: 0.0092 - val_loss: 0.0089 - val_mean_squared_error: 0.0089
Epoch 35/150
135/135 [==============================] - 28s 210ms/step - loss: 0.0088 - mean_squared_error: 0.0088 - val_loss: 0.0089 - val_mean_squared_error: 0.0089
Epoch 36/150
135/135 [==============================] - 28s 211ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0078 - val_mean_squared_error: 0.0078
Epoch 37/150
135/135 [==============================] - 29s 213ms/step - loss: 0.0114 - mean_squared_error: 0.0114 - val_loss: 0.0076 - val_mean_squared_error: 0.0076
Epoch 38/150
135/135 [==============================] - 28s 211ms/step - loss: 0.0092 - mean_squared_error: 0.0092 - val_loss: 0.0069 - val_mean_squared_error: 0.0069
Epoch 39/150
135/135 [==============================] - 28s 211ms/step - loss: 0.0099 - mean_squared_error: 0.0099 - val_loss: 0.0079 - val_mean_squared_error: 0.0079
Epoch 40/15

135/135 [==============================] - 28s 210ms/step - loss: 0.0176 - mean_squared_error: 0.0176 - val_loss: 0.0156 - val_mean_squared_error: 0.0156
Epoch 84/150
135/135 [==============================] - 29s 213ms/step - loss: 0.0170 - mean_squared_error: 0.0170 - val_loss: 0.0107 - val_mean_squared_error: 0.0107
Epoch 85/150
135/135 [==============================] - 29s 213ms/step - loss: 0.0184 - mean_squared_error: 0.0184 - val_loss: 0.0126 - val_mean_squared_error: 0.0126
Epoch 86/150
135/135 [==============================] - 28s 208ms/step - loss: 0.0165 - mean_squared_error: 0.0165 - val_loss: 0.0153 - val_mean_squared_error: 0.0153
Epoch 87/150
135/135 [==============================] - 28s 210ms/step - loss: 0.0194 - mean_squared_error: 0.0194 - val_loss: 0.0146 - val_mean_squared_error: 0.0146
Epoch 88/150
135/135 [==============================] - 28s 210ms/step - loss: 0.0179 - mean_squared_error: 0.0179 - val_loss: 0.0123 - val_mean_squared_error: 0.0123
Epoch 89/15

Epoch 132/150
135/135 [==============================] - 29s 212ms/step - loss: 0.0180 - mean_squared_error: 0.0180 - val_loss: 0.0106 - val_mean_squared_error: 0.0106
Epoch 133/150
135/135 [==============================] - 28s 211ms/step - loss: 0.0178 - mean_squared_error: 0.0178 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 134/150
135/135 [==============================] - 29s 211ms/step - loss: 0.0173 - mean_squared_error: 0.0173 - val_loss: 0.0110 - val_mean_squared_error: 0.0110
Epoch 135/150
135/135 [==============================] - 28s 211ms/step - loss: 0.0173 - mean_squared_error: 0.0173 - val_loss: 0.0106 - val_mean_squared_error: 0.0106
Epoch 136/150
135/135 [==============================] - 28s 209ms/step - loss: 0.0183 - mean_squared_error: 0.0183 - val_loss: 0.0106 - val_mean_squared_error: 0.0106
Epoch 137/150
135/135 [==============================] - 28s 210ms/step - loss: 0.0174 - mean_squared_error: 0.0174 - val_loss: 0.0261 - val_mean_squared_error:

### Evaluación del Modelo

#### Se carga el modelo

In [35]:
model = tf.keras.models.load_model(f'../Models/{dataset_name}_{I}_0.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, None, 216)         206496    
                                                                 
 lstm_6 (LSTM)               (None, 168)               258720    
                                                                 
 dense_6 (Dense)             (None, 280)               47320     
                                                                 
 dense_7 (Dense)             (None, 4)                 1124      
                                                                 
Total params: 513,660
Trainable params: 513,660
Non-trainable params: 0
_________________________________________________________________


In [36]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics='mean_squared_error')

In [37]:
model.save(f'../Models/{dataset_name}_{I}_{0}.h5')

In [38]:
print(f'../Models/{dataset_name}_{I}_{0}.h5')

../Models/DatasetXYZYaw_2022V3_Tuner_0.h5


#### Evaluación con dataset de prueba

In [39]:
%%time
N = 0
for i in range(len(X_test)):
    N+=len(X_test[i])
N=N/len(X_test)

n_batches = np.ceil(N/batch_size)
losses = model.evaluate(dataset_test, steps = n_batches)
K = df_desc[rpm_list[0]][7]-df_desc[rpm_list[0]][3] #Ganancia del actuador
print(f'K={"{:.2f}".format(K)}')
if not type(metrics) == list:
    metrics = [metrics]
for i, l in enumerate(['loss']+metrics):
    print(f'{l}: {"{:.2e}".format(losses[i])} -> {"{:.2f}".format(losses[i]*K)} RPM')

44/44 [==============================] - 7s 152ms/step - loss: 1.3219e-04 - mean_squared_error: 1.3219e-04
K=490.00
loss: 1.32e-04 -> 0.06 RPM
mean_squared_error: 1.32e-04 -> 0.06 RPM
CPU times: user 7.06 s, sys: 27.9 ms, total: 7.09 s
Wall time: 7.24 s


#### Evaluación con 1 trayectoria

In [40]:
window_test = 3000
test_traj_generator  = DataGenerator(X=X_test,  Y=Y_test,  batch_size=1, window=window_test, sequence_out=True, feedback=feedback)

In [41]:
%%time
X, Y = next(test_traj_generator.build_data())
x = X[0]
y = Y[0]
y_pred = []
for i in range(0, len(x)):
    if i == 0:
        x_temp = x[0].reshape(1,  1, X.shape[2])
    elif i<=window:
        x_temp = x[0:i].reshape(1,  i, X.shape[2])
    else:
        x_temp = x[i-window:i].reshape(1,  window, X.shape[2])
    y_temp = model.predict(x_temp)
    y_pred.append(y_temp)
y_pred = np.array(y_pred).reshape(y.shape)

CPU times: user 2min 28s, sys: 13.6 s, total: 2min 42s
Wall time: 2min 21s


#### Visualización con 1 trayectoria

In [42]:
t = np.arange(0, len(y)*Ts, Ts)

NameError: name 'Ts' is not defined

In [ ]:
fig, axs = plt.subplots(Y.shape[2], figsize = (7,15))
for i in range(Y.shape[2]):
    axs[i].plot(t, y_pred[:,i], t, y[:,i])
    axs[i].grid()

In [ ]:
fig, axs = plt.subplots(Y.shape[2], figsize = (7,15))
L1 = 4*len(t)//5
L2 = 5*len(t)//5
for i in range(Y.shape[2]):
    axs[i].plot(t[L1:L2], y_pred[L1:L2,i], t[L1:L2:], y[L1:L2:,i])
    axs[i].grid()

#### Tiempo de inferencia tamaño entrada

In [ ]:
import time

def MSE(y, y_pred):
    return (y-y_pred)**2

window_test = 1000
test_traj_batch = 1000
test_traj_generator  = DataGenerator(X=X_test,  Y=Y_test,  batch_size=test_traj_batch, window=window_test, sequence_out=True)

window_len = []
inf_time = []
loss = []

x, y = next(test_traj_generator.build_data())

In [ ]:
%%time
accum_time = 0
n_iter = 200
test_range = list(set(np.rint(np.logspace(0, np.log10(x.shape[1]-1), num=n_iter, endpoint=True))))
test_range.sort()
n_iter = len(test_range)
for k, i in enumerate(list(map(int,test_range))):
    inf_time_aux = []    # Tiempo de inferencia auxiliar
    loss_aux = []    # Costo Auxiliar
    init_time = time.time()
    for j in range(len(x)):
        x_5 = x[j][0:i].reshape(1,  i, x.shape[2])
        start_time = time.time()
        y_pred = model.predict(x_5)
        finish_time = time.time() - start_time
        inf_time_aux.append(finish_time)
        loss_aux.append(
            np.mean(
                MSE(y[j][i], y_pred)
            )
        )
    
    window_len.append(i)
    inf_time.append(np.mean(inf_time_aux))
    loss.append(np.mean(loss_aux))
    clear_output(wait=True)
    accum_time += time.time()-init_time
    print(f'iter = {k} de {len(test_range)}, i = {i}, execution time = {"{:.2f}".format(np.max(accum_time))}s')

In [ ]:
inf_time_norm = np.array(inf_time)/max(inf_time)
loss_norm = np.array(loss)/max(loss)
performance = 1/(np.array(loss)*np.array(inf_time))
performance = np.array(performance)/max(performance)

In [ ]:
plt.figure()
plt.plot(window_len, inf_time_norm, label='Normalized inference time', alpha=0.7)
plt.plot(window_len, loss_norm, label='Normalized average MSE', alpha=0.7)
plt.plot(window_len, performance, label='Performance=1/(MSE*Time)', alpha=0.7)
plt.xlabel("Window Size")
plt.legend(loc=1)
plt.grid()

print(f'Best window inference time={np.argmin(inf_time_norm)} steps, time={"{:.2f}".format(np.min(inf_time)*1000)} ms')
print(f'Best window MSE loss={np.argmin(loss_norm)} steps, MSE={"{:.2e}".format(np.min(loss))}')
print(f'Best window Performance (MSExTime)={np.argmax(performance)} steps, Value={"{:.2e}".format(np.max(loss)/np.min(inf_time))}')

In [ ]:
plt.figure()
plt.semilogx(window_len, inf_time_norm, label='Normalized inference time', alpha=0.7)
plt.semilogx(window_len, loss_norm, label='Normalized average MSE', alpha=0.7)
plt.semilogx(window_len, performance, label='Performance=1/(MSE*Time)', alpha=0.7)
plt.xlabel("Window Size")
plt.legend(loc=1)
plt.grid()

In [ ]:
plt.figure()
plt.scatter(loss_norm, inf_time_norm, alpha=0.7)